### Rotation parameter in differenct bins
vs mass, vs environment


#### Load catalogs

In [1]:
import matplotlib.pyplot as plt
import pickle
import numpy as np

## fucntions 
def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)


def plot_lambda(catalog, i_early, i_late, i_bad, fn_out='./'):
    import matplotlib.pyplot as plt
    plt.ioff()
    f = plt.figure()
    ax = f.add_subplot(111)
    #for i, val in enumerate(lambdar_arr):
    for i in i_early:
        a = np.asarray(catalog['lambda_arr'][i])
        ax.plot(a, 'r-', alpha=0.5) # Red = Early
    for i in i_late:
        ax.plot(catalog['lambda_arr'][i], 'b-', alpha=0.3) # Red = Early
    
    #plt.xlabel() # in the unit of Reff
    ax.set_title(r"$\lambda _{R}$") 
    ax.set_ylabel(r"$\lambda _{R}$") 
    ax.set_xlabel("["+ r'$R/R_{eff}$'+"]")
    ax.set_xlim(right=9)
    ax.set_ylim([0,1])
    ax.set_xticks([0, 4.5, 9])
    ax.set_xticklabels(["0", "0.5", "1"])
    plt.savefig(fn_out)
    plt.close()
    
def aexp2zred(aexp):
    return [1.0/a - 1.0 for a in aexp]

def zred2aexp(zred):
    return [1.0/(1.0 + z) for z in zred]

def lbt2aexp(lts):
    import astropy.units as u
    from astropy.cosmology import WMAP7, z_at_value
    zreds = [z_at_value(WMAP7.lookback_time, ll * u.Gyr) for ll in lts]
    return [1.0/(1+z) for z in zreds]

def density_map(x, y, ax, sort=True):
    from scipy.stats import gaussian_kde
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy) 
    z /= max(z)

    idx = z.argsort()    
    xx, yy = x[idx], y[idx]
    z = z[idx]
    
    im = ax.scatter(xx, yy, c=z, s=50, edgecolor='')
    return im

In [2]:
clusters = [5427, 36415, 39990, 1605, 10002, 36413, 4466, 74010][0:5]
clu_var=[[]]*len(clusters)

In [3]:
# scatter
from tree import treemodule
import tree.ctutils as ctu

import tree.halomodule as hmo 
import load 

is_gal = False

if is_gal:
    # Galaxy tree
    tree_path = 'GalaxyMaker/Trees/'
else:
    # halo tree
    tree_path = 'halo/Trees/'

nout = 70
history = False

# variables to plot
dtype=[('aexp', '>f4'),('x', '>f8'), ('y', 'f8'), ('z', 'f8'),
        ('rvir', 'f8')]
hdtype=[('nden', 'f8', (4,)), ('mden', 'f8', (4,)),
        ('nbigger', 'f8', (4,)), ('nsmaller', 'f8', (4,)),('mgass', 'f8')]

if history:
    # dervie cluster positions from halo Tree. 
    for i, cluster in enumerate(clusters):
        wdir = '/home/hoseung/Work/data/' + str(cluster).zfill(5) + '/'
        alltrees = treemodule.CTree()
        alltrees.load(filename= wdir + tree_path + 'tree_0_0_0.dat')
        td = alltrees.data

        info = load.info.Info(nout=nout, base=wdir, load=True)
        hh = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info, load=True, is_gal=is_gal)
        id_cluster = hh.data['id'][np.argmax(hh.data['np'])]
        #alltrees = load_pickle(wdir + '/halo/Trees/extended_tree.pickle')

        td_final = td[td['aexp'] == 1.0]
        idx_cluster = td_final['id'][td_final['Orig_halo_id'] == id_cluster]
        main_tree = ctu.extract_main_tree(td, idx_cluster)

        clu_data = np.zeros(len(main_tree), dtype=dtype)

        clu_data['aexp'] = main_tree['aexp'][:,0]    
        clu_data['x'] = main_tree['x'][:,0]
        clu_data['y'] = main_tree['y'][:,0]
        clu_data['z'] = main_tree['z'][:,0]
        clu_data['rvir'] = main_tree['rvir'][:,0]
        clu_var[i] = clu_data
else:
    hh_var=[]
    for i, cluster in enumerate(clusters):
        print(cluster)
        wdir = '/home/hoseung/Work/data/' + str(cluster).zfill(5) + '/'

        info = load.info.Info(nout=nout, base=wdir, load=True)
        hh = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info, load=True, is_gal=False)
        i_cluster = np.argmax(hh.data['np'])
        #alltrees = load_pickle(wdir + '/halo/Trees/extended_tree.pickle')

        # DM halo properties
        clu_data = np.zeros(1, dtype=dtype)

        clu_data['aexp'] = info.aexp
        clu_data['x'] = hh.data[i_cluster]['x'] * 200
        clu_data['y'] = hh.data[i_cluster]['y'] * 200
        clu_data['z'] = hh.data[i_cluster]['z'] * 200
        clu_data['rvir'] = hh.data[i_cluster]['rvir'] * 200
        
        # Galaxy properties
        gg = hmo.Halo(base=wdir, nout=nout, halofinder='HM', info=info, load=True, is_gal=True)
        cd = load_pickle(wdir + 'catalog_GM/' + 'catalog' + str(nout)+'.pickle')
        mstar_cut = 5e9
        i_gal_ok= np.where(cd['mstar'] > mstar_cut)[0]
        gg_data = np.zeros(len(i_gal_ok), dtype=hdtype)
        import utils.sampling as smp
        
        
        for ihal, hal in enumerate(cd['id'][i_gal_ok]):
            ind = np.where(gg.data['id'] == hal)[0]
            for j in range(4):
                scale = 0.5 * (1+j) / 200 / gg.data['rvir'][ind]
                
                # returns a mask array.
                i_close = smp.extract_halos_within(gg.data, ind, scale=scale)
                                
                # number density
                gg_data['nden'][ihal,j] = sum(i_close)
                gg_data['nbigger'][ihal,j] = sum(gg.data['m'] > cd['mstar'][i_gal_ok[ihal]])
                gg_data['nsmaller'][ihal,j] = sum(gg.data['m'] < cd['mstar'][i_gal_ok[ihal]])
                #print(scale, sum(i_close))
                # mass density
                gg_data['mden'][ihal,j] = sum(gg.data['m'][i_close])
        
        clu_var[i] = clu_data
        hh_var.append(gg_data)

5427
36415
39990
1605
10002


In [6]:
def density_estimation(m1, m2):
    from scipy import stats
    X, Y = np.mgrid[min(m1):max(m1):100j, min(m2):max(m2):100j]                                                     
    positions = np.vstack([X.ravel(), Y.ravel()])                                                       
    values = np.vstack([m1, m2])                                                                        
    kernel = stats.gaussian_kde(values)                                                                 
    Z = np.reshape(kernel(positions).T, X.shape)
    return X, Y, Z

#plt.clf()
fig, ax = plt.subplots(3,3)
fig.set_size_inches(16,12)
n_good = 0

plt.ioff()

colors = ['red', 'green', 'blue', 'yellow', 'black']

iscale = 3


cluster_wise = True
lrot, lmgas, lxpos, lypos, lzpos, lmass = [], [], [], [], [],[]
lnden, lmden, lbigger, lsmaller, lsmtobig=[], [], [], [] ,[]


for i, cluster in enumerate(clusters):
    wdir = '/home/hoseung/Work/data/' + str(cluster).zfill(5)
    cd = load_pickle(wdir + '/catalog_GM/' + 'catalog' + str(nout)+'.pickle')

    i_gal_ok=cd['mstar'] > mstar_cut
    
    rot = cd['lambda_r'][i_gal_ok]
    mgas = cd['mgas'][i_gal_ok]
    xpos = cd['xc'][i_gal_ok] / info.pboxsize * 200
    ypos = cd['yc'][i_gal_ok] / info.pboxsize * 200
    zpos = cd['zc'][i_gal_ok] / info.pboxsize * 200
    mass = cd['mstar'][i_gal_ok]
    
    
    lrot.append(rot)
    lmgas.append(mgas)
    lxpos.append(xpos)
    lypos.append(ypos)
    lzpos.append(zpos)
    lmass.append(mass)
    
    # 1. distance[Mpc/h]
    dclu = np.sqrt( np.square(xpos - clu_var[i]['x'][0]) + \
                    np.square(ypos - clu_var[i]['y'][0]) + \
                    np.square(zpos - clu_var[i]['z'][0]))#/clu_var[i]['rvir']
    
    
    #size = mass/1e9
    size=20
    #cc = colors[i]
    cc = (np.log10(mass) - 9) * 20
    
    ylim = 0.8
    
    ax[0,0].scatter(dclu, rot, c=cc, s=size, alpha=0.5, clip_on=True)
    ax[0,0].set_xlabel("distance from the cluster center[Mpc/h]")
    ax[0,0].set_ylim([0,ylim])
    #ax[0,0].set_xlim([-0.2,20])
    ax[0,0].xaxis.major.locator.set_params(nbins=5)

    # 2. mass
    ax[0,1].scatter(np.log10(mass), rot, c=cc, s=size, alpha=0.5)#, cmap=plt.cm.Reds)
    ax[0,1].set_xlabel("log10(mstar/$m_{\odot}$)")
    ax[0,1].set_ylim([0,ylim])

    # 3. gas mass
    ax[0,2].scatter(np.log10(mgas), rot, c=cc, s=size, alpha=0.5)
    ax[0,2].set_xlabel("log10(mgas/$m_{\odot}$)")
    ax[0,2].set_ylim([0,ylim])
    ax[0,2].xaxis.major.locator.set_params(nbins=5)

    # 3-1. gas fraction
    ax[1,2].scatter(np.log10(mgas/mass), rot, c=cc, s=size, alpha=0.5)
    ax[1,2].set_xlabel("log10(mgas/mstar)")
    ax[1,2].set_ylim([0,ylim])

    # 4. nden
    nden = hh_var[i]['nden'][:,iscale]#[0]
    lnden.append(nden)
    ax[1,0].scatter(nden, rot, c=cc, s=size, alpha=0.5, clip_on=True)
    ax[1,0].set_xlabel("# galaxies within {:.1f}Mpc/h".format(0.5*(iscale + 1)))
    #ax[1,0].set_xlim([-2, 100])
    ax[1,0].set_ylim([0,ylim])

    # 5. mden
    mden = hh_var[i]['mden'][:,iscale]#[0]
    lmden.append(mden)
    ax[1,1].scatter(np.log10(mden), rot, c=cc, s=size, alpha=0.5)
    ax[1,1].set_xlabel("total stellar mass within {:.1f}Mpc/h".format(0.5*(iscale + 1)))
    ax[1,1].set_ylim([0,ylim]) 
    #ax[1,1].set_xlim([9.5,12.5]) 
    ax[1,1].xaxis.major.locator.set_params(nbins=4)

    # 6. # bigger
    bigger = hh_var[i]['nbigger'][:,iscale]
    lbigger.append(bigger)
    ax[2,0].scatter(np.log10(bigger), rot, c=cc, s=size, alpha=0.5)
    ax[2,0].set_xlabel("# bigger galaxies within {:.1f}Mpc/h".format(0.5*(iscale + 1)))
    ax[2,0].set_ylim([0,ylim])    
    
    # 7. # smaller    
    smaller = hh_var[i]['nsmaller'][:,iscale]
    lsmaller.append(smaller)
    ax[2,1].scatter(np.log10(smaller), rot, c=cc, s=size, alpha=0.5)
    ax[2,1].set_xlabel("# smaller galaxies within {:.1f}Mpc/h".format(0.5*(iscale + 1)))
    ax[2,1].set_ylim([0,ylim]) 
    
    # 8. # smaller/bigger
    nfrac = smaller/bigger
    lsmtobig.append(nfrac)
    ax[2,2].scatter(np.log10(nfrac), rot, c=cc, s=size, alpha=0.5)
    ax[2,2].set_xlabel("# f smaller/bigger within {:.1f}Mpc/h".format(0.5*(iscale + 1)))
    ax[2,2].set_ylim([0,ylim]) 
    
    fig.suptitle("redshift = {:.1f}".format(info.zred))

#plt.show()
plt.savefig("/home/hoseung/Dropbox/Rotators/clusterwise.svg", format='svg', dpi=1200)

/usr/local/lib/python3.4/dist-packages/IPython/kernel/__main__.py:71: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.4/dist-packages/IPython/kernel/__main__.py:77: RuntimeWarning: divide by zero encountered in log10


In [5]:
import itertools
rot_all = np.fromiter(itertools.chain.from_iterable(lrot), dtype='float')
mgas_all = np.fromiter(itertools.chain.from_iterable(lmgas), dtype='float')
xpos_all = np.fromiter(itertools.chain.from_iterable(lxpos), dtype='float')
ypos_all = np.fromiter(itertools.chain.from_iterable(lypos), dtype='float')
zpos_all = np.fromiter(itertools.chain.from_iterable(lzpos), dtype='float')
mass_all = np.fromiter(itertools.chain.from_iterable(lmass), dtype='float')
nden_all = np.fromiter(itertools.chain.from_iterable(lnden), dtype='float')
mden_all = np.fromiter(itertools.chain.from_iterable(lmden), dtype='float')

ind = np.where(rot_all < 0.15)[0]
X,Y,Z = density_estimation(np.log10(mass_all[ind]), rot_all[ind])
cset = ax[0,1].contour(X, Y, Z, 3)#, colors='k')
    


#plt.savefig("clusterwise.eps", format='eps', dpi=600)
#plt.close()

/usr/local/lib/python3.4/dist-packages/IPython/kernel/__main__.py:71: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.4/dist-packages/IPython/kernel/__main__.py:77: RuntimeWarning: divide by zero encountered in log10


### Mass function

In [10]:
# Mass function
plt.hist(np.log10(mass_all))
plt.show()

## density map

In [7]:
fig, ax = plt.subplots(1)
fig.set_size_inches(8,12)
# compile data


xmin = xall.min()
xmax = xall.max()
ymin = yall.min()
ymax = yall.max()

import scipy.stats as st
# Peform the kernel density estimate
xx, yy = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([xx.ravel(), yy.ravel()])
values = np.vstack([xall, yall])
kernel = st.gaussian_kde(values)
f = np.reshape(kernel(positions).T, xx.shape)


ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
# Contourf plot
cfset = ax.contourf(xx, yy, f, cmap='Blues')
## Or kernel density estimate plot instead of the contourf plot
#ax.imshow(np.rot90(f), cmap='Blues', extent=[xmin, xmax, ymin, ymax])
# Contour plot
cset = ax.contour(xx, yy, f, colors='k')
# Label plot
ax.clabel(cset, inline=1, fontsize=10)
ax.set_xlabel('Y1')
ax.set_ylabel('Y0')

#import seaborn as sns
#sns.kdeplot(data, shade=True, ax=ax[0,1])

#ax[0,1].hist2d(X,Y,bins=40)
#ax[0,1].set_xlabel("log10(mstar)")
#ax[0,1].set_ylim([0,ylim])    
    
plt.show()
plt.close()
#plt.savefig(".eps", format='eps', dpi=30)

NameError: name 'xall' is not defined

##  3D

In [11]:
from mpl_toolkits.mplot3d import Axes3D

#fig, ax = plt.subplots(2,3)
#ax = ax.ravel()



cluster=clusters[0]
for nout in range(37,187):
#for i, cluster in enumerate(clusters[0:1]):
    fig = plt.figure()
    fig.set_size_inches([14,11])
    ax = fig.add_subplot(111, projection='3d')
    wdir = '/home/hoseung/Work/data/' + str(cluster).zfill(5)
    info = load.info.Info(nout=nout, base=wdir, load=True)
    cd = load_pickle(wdir + '/catalog_GM/' + 'catalog' + str(nout)+'.pickle')
   
    i_gal_ok=cd['mstar'] > mstar_cut
    
    rot = cd['lambda_r'][i_gal_ok]
    mgas = cd['mgas'][i_gal_ok]
    mass = (cd['mstar'][i_gal_ok])
    
    xpos = cd['xc'][i_gal_ok] / info.pboxsize * 200
    ypos = cd['yc'][i_gal_ok] / info.pboxsize * 200
    zpos = cd['zc'][i_gal_ok] / info.pboxsize * 200
    
    size = mass/5e8
    #cc = (np.log10(mass) - 9) * 20
    cc = (1-rot) * 200
    
    # 2D map
    #ax[i].scatter(xpos, ypos, c=cc, s=size, alpha=0.5)
    # 3D map
    ax.set_xlim([96, 108])
    ax.set_ylim([54, 70])
    ax.set_zlim([58, 70])
    ax.scatter(xpos, ypos, zs=zpos, c=cc, s=size, alpha=0.5)
    ax.set_title("z= {:.2f}".format(info.zred))
    
    plt.savefig("/home/hoseung/Dropbox/Rotators/3dfigs/" + str(nout) +'.png')

/usr/lib/python3/dist-packages/matplotlib/pyplot.py:412: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_num_figures`).
  max_open_warning, RuntimeWarning)
